In [ ]:
# import necessary libraries

import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve

import matplotlib.pyplot as plt

In [ ]:
predicted_labels = np.load('C:/Users/lunam/Documents/1steMaster/Stage/Data_FinalArrays/Kaggle/Arrays_10GB/predicted_test_labels.npy')
test_labels = np.load('C:/Users/lunam/Documents/1steMaster/Stage/Data_FinalArrays/Kaggle/Arrays_10GB/test_labels_Final.npy')

In [ ]:
def ROC(pred_probs, true_probs, plot = True):
    '''
    This function calculates the ROC-AUC value
    and it also calculates and visualizes the ROC-curve (if plot is true)
    '''
    
    # calculate and print out ROC-AUC value
    ROC_AUC = roc_auc_score(true_probs, pred_probs)
    
    # calculate and plot the ROC-curve
    if plot:
        FPRate, TPRate, Thresh = roc_curve(true_probs, pred_probs)
        plt.figure()
        plt.plot(FPRate, TPRate)
        plt.title('ROC curve')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.show()

    return ROC_AUC

In [ ]:
def PRC(pred_probs, true_probs, plot = True):
    '''
    Calculate the aupr value = the area under the precision-recall curve
    and plot the precision-recall curve (if plot is true)
    '''
    
    # calculate the precision-recall curve
    Precision, Recall, Thresh = precision_recall_curve(true_probs, pred_probs)
    Precision = np.fliplr([Precision])[0]  # so the array is increasing (you won't get negative AUC)
    Recall = np.fliplr([Recall])[0]  # so the array is increasing (you won't get negative AUC)
    AUPR = np.trapz(Precision, Recall)
      
    
    # plot the precision-recall curve
    if plot:
        plt.figure()
        plt.plot(Precision, Recall)
        plt.title('Precision-Recall curve')
        plt.xlabel('Precision')
        plt.ylabel('Recall')
        plt.show()
    
    return AUPR

In [ ]:
def ConfusionMatrix(pred_probs, true_probs, threshold_confusion = 0.5):
    '''
    Calculate and print out the confusion matrix
    The values for the confusion matrix are also returned
    A standard threhsold of 0.5 is used to calculate the confusion matrix
    '''
    # print out the threshold that is used for the confusion matrix
    print("Confusion matrix with a used threshold of {} for the positive class:".format(threshold_confusion))
    
    # turn the predicted probability maps into binary outputs according to the threshold
    thresh_pred_probs = np.empty((pred_probs.shape[0]))
    for i in range(pred_probs.shape[0]):
        if pred_probs[i] >= threshold_confusion:
            thresh_pred_probs[i] = 1
        else:
            thresh_pred_probs[i] = 0
    
    # calculate the confusion-matrix and print it out
    TN, FP, FN, TP = confusion_matrix(true_probs, thresh_pred_probs).ravel()
    print('Amount of true positives: {}'.format(TP))
    print('Amount of false positives: {}'.format(FP))
    print('Amount of true negatives: {}'.format(TN))
    print('Amount of false negatives: {}'.format(FN))
    
    return TN, FP, FN, TP

In [ ]:
def Metrics(tn, fp, fn, tp):
    '''Print out some metrics like the accuracy, specificity, sensitivity and precision'''
    
    print('Some metrics:')
    
    # print out accuracy
    accuracy = 0
    if (tn+fp+fn+tp) != 0:
        accuracy = (tp+tn)/(tn+fp+fn+tp)
    print('Accuracy: {}'.format(accuracy))
    
    # print out specificity
    specificity = 0
    if (tn+fp) != 0:
        specificity = tn/(tn+fp)
    print('Specificity: {}'.format(specificity))
    
    # print out sensitivity
    sensitivity = 0
    if (tp+fn) != 0:
        sensitivity = tp/(tp+fn)
    print('Sensitivity: {}'.format(sensitivity))
    
    # print out precision
    precision = 0
    if (tp+fp) != 0:
        precision = tp/(tp+fp)
    print('Precision: {}'.format(precision))


In [ ]:
# evaluation of the model
def Evaluate(pred_labels, true_labels):
    '''
    Evaluation of the network, based on comparisson between the true and predicted outputs
    '''
    print('Evaluating the model...')
    
    # creat a 1D numpy array with the predicted and true outputs
    pred_pos_probs = pred_labels[:,0]
    true_pos_probs = true_labels[:,0]

    # print out ROC-AUC value and plot the ROC-curve
    ROC_AUC = ROC(pred_pos_probs, true_pos_probs)
    print('Area under the ROC curve: {}'.format(ROC_AUC))

    # print out area under the precision-recall curve, AUPR value
    AUPR = PRC(pred_pos_probs, true_pos_probs)
    print('Area under Precision-Recall curve: {}'.format(AUPR))
    
    # print out confusion matrix
    TN, FP, FN, TP = ConfusionMatrix(pred_pos_probs, true_pos_probs)
    
    # print out some metrics
    Metrics(TN, FP, FN, TP)

In [ ]:
Evaluate(predicted_labels , test_labels)